In [1]:
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import dateutil
import analysis

packets = analysis.parse_packet_file('data/all.txt')
packets += analysis.parse_packet_file('data/temp_basals.txt')

# Select valid packets of a particular length
packets = filter(lambda x: x.is_valid() and x.body_len == 10, packets)

# Uncomment if you want to work with less data (faster while developing)
#packets = packets[0:100]

In [2]:
cracked_bits_dict = analysis.build_bitdiff_dictionary(packets)

In [3]:
[key for key in cracked_bits_dict.keys() if len(key.split(':')) == 1]

['116',
 '111',
 '113',
 '112',
 '31',
 '30',
 '37',
 '36',
 '35',
 '34',
 '115',
 '114',
 '117']

In [4]:
print cracked_bits_dict['36:80:115']

36:80:115: msg=[0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 128, 0, 0, 0, 16, 0], crc=[129, 61]


In [5]:
[key for key in cracked_bits_dict.keys() if len(key.split(':')) == 2]

['30:79',
 '30:116',
 '30:31',
 '34:35',
 '34:37',
 '34:36',
 '79:116',
 '79:112',
 '79:111',
 '78:114',
 '78:110',
 '58:59',
 '35:36',
 '80:112',
 '116:117',
 '77:116',
 '35:37',
 '58:82',
 '31:79',
 '111:113',
 '111:112',
 '111:114',
 '111:115',
 '111:117',
 '80:113',
 '76:108',
 '112:113',
 '80:127',
 '59:117',
 '115:116',
 '115:117',
 '112:117',
 '112:116',
 '77:109',
 '79:113',
 '112:115',
 '112:114',
 '36:83',
 '36:80',
 '113:116',
 '113:117',
 '113:114',
 '113:115',
 '114:115',
 '114:117',
 '114:116',
 '35:82',
 '35:80',
 '83:115',
 '83:117',
 '83:116',
 '83:113',
 '83:112',
 '37:117',
 '83:111',
 '111:116',
 '80:111',
 '80:116',
 '80:114',
 '80:115',
 '35:116',
 '35:117',
 '34:117',
 '34:116',
 '36:113',
 '36:112',
 '36:114',
 '36:117',
 '36:116',
 '36:37']

In [6]:
[key for key in cracked_bits_dict.keys() if len(key.split(':')) == 3]

['78:82:83',
 '79:112:113',
 '79:112:114',
 '79:112:115',
 '36:78:82',
 '31:37:82',
 '31:37:80',
 '80:114:115',
 '79:80:116',
 '79:80:115',
 '79:80:114',
 '79:80:112',
 '79:80:111',
 '30:31:116',
 '35:80:127',
 '80:116:127',
 '35:80:117',
 '35:80:116',
 '79:112:116',
 '79:112:117',
 '83:113:116',
 '83:113:117',
 '83:113:114',
 '83:113:115',
 '37:58:82',
 '80:116:117',
 '114:115:117',
 '36:111:114',
 '36:111:115',
 '36:111:116',
 '78:110:111',
 '35:81:114',
 '35:36:111',
 '35:36:113',
 '35:82:111',
 '35:82:113',
 '35:82:114',
 '35:82:116',
 '36:83:116',
 '36:80:127',
 '111:112:113',
 '111:112:115',
 '78:109:110',
 '35:58:59',
 '76:108:113',
 '80:112:114',
 '80:112:115',
 '80:112:116',
 '80:112:117',
 '36:79:82',
 '36:79:80',
 '35:83:111',
 '35:83:112',
 '35:83:114',
 '77:116:117',
 '77:79:109',
 '36:78:114',
 '108:109:110',
 '112:116:117',
 '111:113:117',
 '111:113:116',
 '111:113:115',
 '111:113:114',
 '34:78:81',
 '58:83:117',
 '80:114:116',
 '35:79:82',
 '80:115:117',
 '80:115:116',
